In [1]:
# Phu, Andrea and Watcher
# 2018 Spring
import torch
import torch.nn as nn
import torch.autograd as autograd
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
from torch import FloatTensor, LongTensor
import numpy as np
import pandas as pd
import time
import os
import pickle
import string
import torch.utils.data as data_utils
import psutil
from random import shuffle
from sklearn.utils import shuffle as skshuffle
import random

torch.manual_seed(44)
random.seed(44)
np.random.seed(44)

## We first read in the data

In [2]:
processed_data_path = '/data/Dropbox/judge_embedding_data_sp18'
all_data_save_path = os.path.join(processed_data_path,"finalized_all_data","all_data_dict.pkl")
all_data_support_save_path = os.path.join(processed_data_path,"finalized_all_data","all_data_dict_support.pkl")
all_data_df_save_path = os.path.join(processed_data_path,"finalized_all_data","all_data_df.h5")
finished_embedding_folder_path = os.path.join(processed_data_path,'finished_judge_embedding')


In [3]:
all_data_dict = pickle.load(open(all_data_save_path,"rb"))

In [4]:
all_data_dict.keys()

dict_keys(['README', 'data_df', 'judge_id_to_index', 'judge_index_to_id', 'topic_glove_emb', 'citation_emb', 'case_to_citation_emb_index', 'citation_name_to_emb_index'])

In [5]:
topic_glove_emb = all_data_dict['topic_glove_emb'][0]
judgeId2Index = all_data_dict['judge_id_to_index'][0]
judgeIndex2Id = all_data_dict['judge_index_to_id'][0]
all_data_df = all_data_dict['data_df']
all_data_df

,caseid,year,middle_part,opinion_text,songername,judgeidentificationnumber,topic,geniss,opinion_vector,judge_embed_index,Circuit,date,affirmdummy,judge_decision
0,XB0PMNQNB5G0,1891,contentMajOp,"[0, 202, 1268, 13, 0, 4600, 3, 0, 247954, 128,...","PARDEE, DON A.",1830.0,Civil Procedure,4.0,"[-0.07445931306292275, 0.05843820821632856, -0...",0,5,1891-12-07,0.0,0
1,XFL757,1891,contentMajOp,"[15, 116, 4, 9238, 7, 22547, 3, 4092, 1219, 0,...","BROWN, HENRY BILLINGS",284.0,Civil Procedure,4.0,"[-0.07101038203845275, 0.029874279749715626, -...",1,6,1891-10-06,0.0,0
2,XB0PMRQNB5G0,1891,contentMajOp,"[215, 6, 68193, 3527, 255, 9670, 5, 12375, 3, ...","PARDEE, DON A.",1830.0,Contracts,7.0,"[-0.059691082627907024, 0.10039284279069777, -...",0,5,1891-12-07,0.0,0
3,XFL7KI,1891,contentMajOp,"[5, 0, 935, 4951, 5, 8151, 0, 4490, 3, 0, 305,...","LOCKE, JAMES WILLIAM",1410.0,Civil Procedure,4.0,"[-0.13210430623456784, 0.12352108086419746, -0...",2,5,1891-11-27,0.0,0
4,XB0OI7QNB5G0,1891,contentMajOp,"[5214, 10, 894, 2091, 4297, 21, 0, 14485, 1268...","NELSON, SUSAN RICHARD",3339.0,NaN,NaN,"[-0.09034438144397865, 0.06449958495506274, -0...",3,8,NaN,0.0,0
5,XB0PHFQNB5G0,1891,contentMajOp,"[0, 377, 202, 3, 14963, 386, 92, 3, 745, 98, 0...","PARDEE, DON A.",1830.0,Torts,7.0,"[-0.06601334912248623, 0.019215826325411318, -...",0,5,1891-11-27,0.0,0
6,XB0PJRQNB5G0,1891,contentMajOp,"[202, 1268, 21, 0, 14485, 38, 14, 60389, 187, ...","BRUCE, JOHN",292.0,Corporate Law,7.0,"[-0.11053901197255082, 0.10235485285668748, -0...",4,5,1891-12-07,1.0,1
7,XFL6PE,1891,contentMajOp,"[15, 116, 4, 18428, 0, 305, 4, 0, 1399, 202, 3...","BROWN, HENRY BILLINGS",284.0,Civil Procedure,4.0,"[-0.05526258051118215, 0.049538111821086264, -...",1,6,1891-10-06,0.0,0
8,XFL6LS,1891,contentMajOp,"[3, 133, 10550, 10, 1609, 400000, 0, 58, 84, 4...","SAGE, GEORGE READ",2089.0,Patent Law,7.0,"[-0.09545400352665524, 0.10511163229642662, -0...",5,6,1891-10-06,0.0,0
9,XB0PNFQNB5G0,1891,contentMajOp,"[0, 2536, 3, 323, 1853, 128, 3, 50, 196, 7091,...","PARDEE, DON A.",1830.0,Mortgages & Liens,7.0,"[-0.08174452291734276, 0.08172237472717521, -0...",0,5,1891-12-07,1.0,1


# We will remove federal court cases 

In [45]:
all_data_df_no_federal = all_data_df[all_data_df.Circuit != 0]
all_data_df_no_federal

,caseid,year,middle_part,opinion_text,songername,judgeidentificationnumber,topic,geniss,opinion_vector,judge_embed_index,Circuit,date,affirmdummy,judge_decision
0,XB0PMNQNB5G0,1891,contentMajOp,"[0, 202, 1268, 13, 0, 4600, 3, 0, 247954, 128,...","PARDEE, DON A.",1830.0,Civil Procedure,4.0,"[-0.07445931306292275, 0.05843820821632856, -0...",0,5,1891-12-07,0.0,0
1,XFL757,1891,contentMajOp,"[15, 116, 4, 9238, 7, 22547, 3, 4092, 1219, 0,...","BROWN, HENRY BILLINGS",284.0,Civil Procedure,4.0,"[-0.07101038203845275, 0.029874279749715626, -...",1,6,1891-10-06,0.0,0
2,XB0PMRQNB5G0,1891,contentMajOp,"[215, 6, 68193, 3527, 255, 9670, 5, 12375, 3, ...","PARDEE, DON A.",1830.0,Contracts,7.0,"[-0.059691082627907024, 0.10039284279069777, -...",0,5,1891-12-07,0.0,0
3,XFL7KI,1891,contentMajOp,"[5, 0, 935, 4951, 5, 8151, 0, 4490, 3, 0, 305,...","LOCKE, JAMES WILLIAM",1410.0,Civil Procedure,4.0,"[-0.13210430623456784, 0.12352108086419746, -0...",2,5,1891-11-27,0.0,0
4,XB0OI7QNB5G0,1891,contentMajOp,"[5214, 10, 894, 2091, 4297, 21, 0, 14485, 1268...","NELSON, SUSAN RICHARD",3339.0,NaN,NaN,"[-0.09034438144397865, 0.06449958495506274, -0...",3,8,NaN,0.0,0
5,XB0PHFQNB5G0,1891,contentMajOp,"[0, 377, 202, 3, 14963, 386, 92, 3, 745, 98, 0...","PARDEE, DON A.",1830.0,Torts,7.0,"[-0.06601334912248623, 0.019215826325411318, -...",0,5,1891-11-27,0.0,0
6,XB0PJRQNB5G0,1891,contentMajOp,"[202, 1268, 21, 0, 14485, 38, 14, 60389, 187, ...","BRUCE, JOHN",292.0,Corporate Law,7.0,"[-0.11053901197255082, 0.10235485285668748, -0...",4,5,1891-12-07,1.0,1
7,XFL6PE,1891,contentMajOp,"[15, 116, 4, 18428, 0, 305, 4, 0, 1399, 202, 3...","BROWN, HENRY BILLINGS",284.0,Civil Procedure,4.0,"[-0.05526258051118215, 0.049538111821086264, -...",1,6,1891-10-06,0.0,0
8,XFL6LS,1891,contentMajOp,"[3, 133, 10550, 10, 1609, 400000, 0, 58, 84, 4...","SAGE, GEORGE READ",2089.0,Patent Law,7.0,"[-0.09545400352665524, 0.10511163229642662, -0...",5,6,1891-10-06,0.0,0
9,XB0PNFQNB5G0,1891,contentMajOp,"[0, 2536, 3, 323, 1853, 128, 3, 50, 196, 7091,...","PARDEE, DON A.",1830.0,Mortgages & Liens,7.0,"[-0.08174452291734276, 0.08172237472717521, -0...",0,5,1891-12-07,1.0,1


## We will do a normalization of doc vectors, using center by topic-year
## That means for each doc vector, we reduce from it the topic-year vector it is related to.
## E.g. if a doc vector is from 1981 and topic of civil procedure, then we reduce it by the average vector of all vectors that are from 1981 and of topic civil procedure.

In [7]:
def get_year_topic_dict(df):
    # given all data df, give the year-topic average vectors
    # in the form of a year-topic dictionary
    # key is year-topic string, and value is a 300 dim vector
    # which is the average of all the cases with that year and topic
    
    yearTopicDict = {}
    yearTopicCount = {}
    
    n_instance = df.shape[0]
    for i in range(n_instance):
        case = df.iloc[i]
        year = case['year']
        topic = case['topic']
        yearTopic = str(year)+"-"+str(topic)
        opinion_vector = case['opinion_vector']
        
        if yearTopic not in yearTopicDict:
            yearTopicDict[yearTopic] = np.zeros(300) + opinion_vector
            yearTopicCount[yearTopic] = 1
        else:
            yearTopicDict[yearTopic] += opinion_vector
            yearTopicCount[yearTopic] += 1
            
    # now we accumulated all the cases
    # we do an average for each dictionary entry
    for k in yearTopicDict.keys():
        yearTopicDict[k] /= yearTopicCount[k]
        
    return yearTopicDict

def add_centered_vec_to_df(df,year_topic_dict,verbose=0):
    starttime = time.time()
    df['centered_opinion_vec'] = None
    n_instance = df.shape[0]
    for i in range(n_instance):
        if verbose and (i==2 or i%int(n_instance/20)==0 ) :
            print(i,time.time()-starttime)
        case = df.iloc[i]
        year = case['year']
        topic = case['topic']
        yearTopic = str(year)+"-"+str(topic)
        meanTopicYear = year_topic_dict[yearTopic]
        opinion_vector = case['opinion_vector']    
        centered_vec = opinion_vector - meanTopicYear
        
        df.iat[i,df.columns.get_loc('centered_opinion_vec')] = centered_vec
    

In [8]:
year_topic_dict = get_year_topic_dict(all_data_df_no_federal)

In [9]:
add_centered_vec_to_df(all_data_df_no_federal,year_topic_dict,verbose=1)

/data/Dropbox/judge_embedding_data_sp18/condaVirtualEnvs/judge_emb/lib/python3.6/site-packages/ipykernel/__main__.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0 5.158310890197754
2 5.159435033798218
14426 8.852142095565796
28852 12.635518789291382
43278 17.327725410461426
57704 22.65861487388611
72130 26.468050956726074
86556 31.9654643535614
100982 35.765554904937744
115408 40.44091534614563
129834 44.36641049385071
144260 49.74828028678894
158686 53.43113136291504
173112 57.22474408149719
187538 61.95547342300415
201964 65.5227403640747
216390 70.1973648071289
230816 73.9133985042572
245242 79.29024720191956
259668 83.71388053894043
274094 88.08116698265076
288520 92.89327311515808


## Now we will convert the data into numpy matrix and get ready for training.
## we first split the data

In [10]:


def train_val_test_split(data_df,number_judges,train_ratio=0.8,val_ratio=0.1,verbose=0,toshuffle=True):
    starttime= time.time()
    sorted_all_data = data_df.sort_values(by='judge_embed_index')
    train_indexes = []
    val_indexes = []
    test_indexes = []
    currentiloc = 0
    for judge_index in range(number_judges):
        if verbose and judge_index%500 == 0:
            print(judge_index,time.time()-starttime)
        
        cases_of_this_judge = sorted_all_data.loc[sorted_all_data['judge_embed_index'] == judge_index]
        number_cases = cases_of_this_judge.shape[0]
        n_of_train = int(number_cases*train_ratio)
        n_of_val = int(number_cases*val_ratio)
        
        nextiloc = currentiloc+number_cases
        
        indexes = [i for i in range(currentiloc, nextiloc)]
        shuffle(indexes)
        
        train_indexes += indexes[:n_of_train]
        val_indexes += indexes[n_of_train:n_of_train+n_of_val]
        test_indexes += indexes[n_of_train+n_of_val:]
        
        currentiloc = nextiloc
    return skshuffle(data_df.loc[train_indexes]),skshuffle(data_df.loc[val_indexes]),skshuffle(data_df.loc[test_indexes])

In [11]:
data_train, data_val, data_test = train_val_test_split(all_data_df_no_federal,2099,verbose=1)

0 0.22924232482910156
500 0.6718382835388184
1000 1.0849401950836182
1500 1.5037636756896973
2000 1.867598056793213


/data/Dropbox/judge_embedding_data_sp18/condaVirtualEnvs/judge_emb/lib/python3.6/site-packages/ipykernel/__main__.py:29: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike


In [35]:
data_train = data_train.dropna()
data_val = data_val.dropna()
data_test = data_test.dropna()

## Now we get opinion vector, ruling label and topic vector to make data into X, y numpy matrix forms

In [36]:
data_train.shape

(219266, 15)

In [37]:
data_train

,caseid,year,middle_part,opinion_text,songername,judgeidentificationnumber,topic,geniss,opinion_vector,judge_embed_index,Circuit,date,affirmdummy,judge_decision,centered_opinion_vec
263151,XF6H54,2004.0,contentMajOp,"[19073, 1128, 400000, 400000, 5, 3992, 27563, ...","RILEY, WILLIAM J.",2891.0,Admiralty & Maritime,7.0,"[-0.08793258939058213, 0.0475965697630039, -0....",1935.0,8.0,2004-07-28,0.0,0.0,"[-0.017251553566165326, -0.007013154413802694,..."
35447,X4A2E7,1931.0,contentMajOp,"[3, 0, 104, 112, 202, 10, 132, 5777, 4, 27520,...","WILBUR, CURTIS D.",2577.0,Agency,7.0,"[-0.06011545870583058, 0.06848046764575985, -0...",318.0,9.0,1931-05-08,1.0,1.0,"[0.024110082705073886, -0.017175822546396824, ..."
219203,X39QGC,1995.0,contentMajOp,"[5, 18527, 270, 9686, 46, 0, 2250, 2929, 25, 7...","ALTIMARI, FRANK X.",35.0,Government,4.0,"[-0.0793168280726737, 0.007910093009926957, -0...",1370.0,2.0,1995-09-11,1.0,1.0,"[0.005009815198643672, -0.050203462862273354, ..."
230812,XACBS0,1998.0,contentMajOp,"[5, 16893, 1317, 1574, 0, 377, 2437, 1949, 162...","ANDERSON, R. LANIER, III",48.0,Civil Rights,2.0,"[-0.07156609695317111, 0.06845639294605789, -0...",1159.0,11.0,1998-08-04,0.0,0.0,"[0.016995925483853563, -0.003441754114261217, ..."
38160,X3IR5M,1933.0,contentMajOp,"[853, 1219, 42, 0, 5064, 15, 243, 39, 32, 19, ...","CHASE, HARRIE B.",413.0,Immigration & Naturalization,9.0,"[-0.06927142857227905, 0.050887525810493314, -...",317.0,2.0,1933-07-25,0.0,0.0,"[0.03057982468375657, -0.011391606029746926, -..."
256553,X6B7FB,2003.0,contentConOp,"[3, 11686, 527, 2748, 0, 439, 1276, 5, 18245, ...","SCIRICA, ANTHONY J.",2128.0,Government,4.0,"[-0.11941561723744287, 0.03669194068493148, -0...",1422.0,3.0,2003-03-19,1.0,1.0,"[-0.0335144223969536, -0.02725149230822138, 0...."
17161,XFLUVQ,1917.0,contentMajOp,"[995, 13851, 2399, 106, 95, 15, 12, 1219, 42, ...","DENISON, ARTHUR C.",602.0,Transportation Law,9.0,"[-0.09345750522866722, 0.08339518608477484, -0...",167.0,6.0,1917-10-02,1.0,1.0,"[0.0015591640604173396, 0.011669034114523413, ..."
8390,XFKLOG,1907.0,contentDisOp,"[22182, 6186, 4, 1625, 192, 30437, 47, 1977, 8...","PHILIPS, JOHN FINIS",1877.0,Transportation Law,9.0,"[-0.09732703148567494, 0.04374137648632252, -0...",88.0,8.0,1907-11-25,0.0,1.0,"[0.0027580072572315206, -0.01952662734120965, ..."
211971,X37PRN,1994.0,contentMajOp,"[6109, 2427, 3219, 4297, 7, 1034, 137, 1553, 6...","HEANEY, GERALD",1015.0,Social Security,7.0,"[-0.13094414730745163, 0.09808817539762052, 0....",803.0,8.0,1994-01-25,1.0,1.0,"[-0.030243955157474273, 0.015741186397642187, ..."
67195,X3CQBG,1950.0,contentMajOp,"[6004, 4, 1287, 0, 451, 3, 0, 693, 202, 14, 40...","KALODNER, HARRY E.",1221.0,Tax & Accounting,7.0,"[-0.1243745507185634, 0.06096408510578844, -0....",406.0,3.0,1950-11-16,0.0,0.0,"[-0.01703913108855694, -0.01021261218110641, -..."


In [50]:
def df_to_Tensor(df,topic_glove_emb,verbose=0):
    # use this to convert a dataframe to torch tensor
    feature_dim = 300+300+2
    X = np.zeros((df.shape[0],feature_dim))
    y = np.zeros(df.shape[0])
    
    for i in range(df.shape[0]):
        if verbose and i%10000==0:
            print(i)
        
        data_entry = df.iloc[i]
        
        X[i,:300] = data_entry['centered_opinion_vec']
        topic = data_entry['topic']
        topic = str.lower(str(topic)) 
        
        if topic not in topic_glove_emb: # deal with any unknown topic
            topic = "<UNK>"
            
        X[i,300:600] = topic_glove_emb[topic]
        decision = int(data_entry['judge_decision'])
        X[i,600+decision] = 1 # one hot representation for judge decision
        y[i] = data_entry['judge_embed_index']
        
    return FloatTensor(X),LongTensor(y)

In [51]:
X_train, y_train = df_to_Tensor(data_train,topic_glove_emb,1)
X_val, y_val = df_to_Tensor(data_val,topic_glove_emb,1)
X_test, y_test = df_to_Tensor(data_test,topic_glove_emb,1)


0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
0
10000
20000
0
10000
20000


## Now we have X_train, y_train and in Tensor form, we can now create the model and start our training

In [52]:
BATCH_SIZE = 128
train_dataset = data_utils.TensorDataset(data_tensor=X_train,target_tensor=y_train)
train_loader = data_utils.DataLoader(dataset=train_dataset,batch_size=BATCH_SIZE,shuffle=True)


In [53]:
class Judge_emb_model(nn.Module):
    def __init__(self, input_dim, hidden_layer_dim, embedding_dim, num_judges):
        super(Judge_emb_model,self).__init__()
        # input is m x D
        self.linear1 = nn.Linear(input_dim,hidden_layer_dim) # D x H 
        self.dropout1 = nn.Dropout(p=0.5)
        self.linear2 = nn.Linear(hidden_layer_dim,hidden_layer_dim) # H x H
        self.dropout2 = nn.Dropout(p=0.5)
        
        self.judge_embedding = nn.Linear(embedding_dim,num_judges) # H x J
        # the output is m x J
        
        self.init_weights()

    def forward(self, X):
        out = F.relu(self.linear1(X))
        out = self.dropout1(out)
        out = F.relu(self.linear2(out))
        out = self.dropout2(out)
        out = self.judge_embedding(out)
        
        # now we have m x J matrix, for m data points, we can do log softmax
        log_prob = F.log_softmax(out,dim=1)
        return log_prob # for each opinion data, this is probability of which judge writes this opinion
    
    def init_weights(self):
        linear_layers = [self.linear1,self.linear2,self.judge_embedding]
        for layer in linear_layers:
            layer.weight.data.normal_(0.0,0.1)


In [54]:
INPUT_DIM = 602
HIDDEN_DIM = 300
EMBED_DIM = 300
number_judges = 2099
model = Judge_emb_model(input_dim=INPUT_DIM,hidden_layer_dim=HIDDEN_DIM,
                        embedding_dim=EMBED_DIM,num_judges=number_judges)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=0.005)

In [55]:
N_EPOCH = 8
TRAIN_SIZE = train_dataset.data_tensor.shape[0]
print("Training data size",TRAIN_SIZE)
train_losses = []
val_losses = []

X_val_var = Variable(X_val)
y_val_var = Variable(y_val)
model.eval()
y_pred_val = model.forward(X_val_var)
val_loss = criterion(y_pred_val,y_val_var)
print("initial val loss",val_loss.data[0])
startTime = time.time()
model.train()

for i_epoch in range(N_EPOCH):
    epoch_train_loss = 0
    num_batches_per_epoch = int(TRAIN_SIZE/BATCH_SIZE)
    for i_batch,(X_batch, y_batch) in enumerate(train_loader):
        optimizer.zero_grad()
        
        X_var, y_var = Variable(X_batch),Variable(y_batch)
        
        y_pred = model.forward(X_var)
        loss = criterion(y_pred,y_var)
        loss.backward()
        
        optimizer.step()
#         if i_batch % 2000 == 0:
#             print(i_epoch,i_batch,loss.data[0])
        epoch_train_loss += loss.data[0]
        
    # after each epoch
    
    X_val_var = Variable(X_val)
    y_val_var = Variable(y_val)
    model.eval()
    y_pred_val = model.forward(X_val_var)
    val_loss = criterion(y_pred_val,y_val_var)
    ave_train_loss = epoch_train_loss/num_batches_per_epoch
    print("epoch",i_epoch,"ave_train_loss",
          ave_train_loss,"validation loss:",val_loss.data[0],time.time()-startTime)
    val_losses.append(val_loss.data[0])
    train_losses.append(ave_train_loss)
    model.train()

Training data size 219266
initial val loss 8.94957447052002
epoch 0 ave_train_loss 6.43345693559307 validation loss: 6.330691337585449 85.6988205909729
epoch 1 ave_train_loss 6.366861349229429 validation loss: 6.336966037750244 213.3677864074707
epoch 2 ave_train_loss 6.3665704880272935 validation loss: 6.340359210968018 348.9057421684265
epoch 3 ave_train_loss 6.367439736000502 validation loss: 6.346747875213623 506.65223813056946
epoch 4 ave_train_loss 6.364888415305989 validation loss: 6.3337016105651855 649.5711059570312
epoch 5 ave_train_loss 6.362532040840412 validation loss: 6.343315124511719 789.636741399765
epoch 6 ave_train_loss 6.364365352223091 validation loss: 6.3473381996154785 945.5797636508942
epoch 7 ave_train_loss 6.362505556196758 validation loss: 6.340451717376709 1096.8800003528595


In [56]:
trained_emb = model.judge_embedding.weight.data.numpy()

In [57]:
trained_emb.shape

(2099, 300)

In [58]:
# trained_emb_path = os.path.join(finished_embedding_folder_path,"trained_emb_May1.pkl")
trained_emb_path = os.path.join(finished_embedding_folder_path,"no_federal_trained_emb_May17.pkl")
pickle.dump(trained_emb,open(trained_emb_path,"wb"))